# EV Clustering

In [1]:
import pandas as pd
import numpy as np
from IMPORT_DATAFRAME_JSON_HDF5 import *

In [2]:
%config InlineBackend.figure_format = 'retina'

## Import jsons/hdf5

In [3]:
data = import_trasient_from_file("jsons/")
data = map_transients_to_PQ_data(data, "HDF5", 20, ['P'])

100% (1370 of 1370) |#####################| Elapsed Time: 0:00:17 Time: 0:00:17
100% (1882 of 1882) |#####################| Elapsed Time: 0:05:11 Time: 0:05:11


## Explore the data

This chapter should give a overview of the used dataset. Therefore step one is to study the header of the dataset. Here the column *'begin_timestamp_float'* can be used as a index, make our're events easier to find and to compare with other data sources. 

In [4]:
length = len(data)
print("Number of data points: {}".format(length))
data.head(1)

Number of data points: 1882


,begin_index,begin_timestamp_string,filtered_signal,phase_num,raw_signal_current,raw_signal_voltage,three_first_peaks,three_first_peaks_index,transient_rise_gradient,P
begin_timestamp_float,,,,,,,,,,
1.491027e+09,701,01-04-2017_08:01:41_529732,"[7.9862e-06, 0.0259479208, -0.0389019368, 7.97...",2,"[0.6561279297, 0.7019042969, 0.7629394531, 0.7...","[289.647277832, 289.6991882324, 289.8029785156...","[26.198, 4.360, 2.076]","[754, 884, 1015]",245806.997513,"[105.77301788330078, 104.60313415527344, 105.3..."


In [5]:
transient_overview = pd.read_csv('transientlist.csv', sep=',', header=None)
transient_overview[0] = transient_overview[0].apply(convert_to_datetime)
len(transient_overview)

# output: 1370

1370

In [6]:
transient_overview.sort_values(0).head(1)

,0,1,2,3
733,2017-01-04 08:01:41.528330,7681,[2],26.198


In [7]:
data.sort_values('begin_timestamp_string').head(1)

,begin_index,begin_timestamp_string,filtered_signal,phase_num,raw_signal_current,raw_signal_voltage,three_first_peaks,three_first_peaks_index,transient_rise_gradient,P
begin_timestamp_float,,,,,,,,,,
1.491027e+09,701,01-04-2017_08:01:41_529732,"[7.9862e-06, 0.0259479208, -0.0389019368, 7.97...",2,"[0.6561279297, 0.7019042969, 0.7629394531, 0.7...","[289.647277832, 289.6991882324, 289.8029785156...","[26.198, 4.360, 2.076]","[754, 884, 1015]",245806.997513,"[105.77301788330078, 104.60313415527344, 105.3..."


## Preprossing Data Seb

In [8]:
#for a in range(0, len(data)):
#    plt.plot(data.iloc[a]['P'])
    

data['charging_status']=0
data['P_mean']=0
data['P_delta'] = 0
data['P_Plugin']=0
data['P_Unplug'] = 0
for row in range(0, len(data)):
    # Getting the mean power before during and after the timestamp (-T, T)
    # Defining charging_status as 1 for avg(P)>300W else 0 and writing it to a new column
    P_mean = sum(data.iloc[row]['P']) / float(len(data.iloc[row]['P']))
    data.set_value(data.index[row],'P_mean', P_mean)
    if P_mean>300:
        data.set_value(data.index[row],'charging_status',int(1))
    else:
       data.set_value(data.index[row],'charging_status', int(0))
     # Defining if a new car is plugged in during the appearance of the transient
    P_mean_before = sum((data.iloc[row]['P'])[:10]) / float(len((data.iloc[row]['P'])[:10])) 
    P_mean_after = sum((data.iloc[row]['P'])[(len(data.iloc[row]['P'])-10):]) / float(len((data.iloc[row]['P'])[(len(data.iloc[row]['P'])-10):])) 
    P_delta = P_mean_after - P_mean_before
    data.set_value(data.index[row],'P_delta',P_delta)
    if P_delta > 2000:
        data.set_value(data.index[row],'P_Plugin',int(1))
    elif P_delta < -2000:
        data.set_value(data.index[row],'P_Unplug',int(1))

In [9]:
data.head()

,begin_index,begin_timestamp_string,filtered_signal,phase_num,raw_signal_current,raw_signal_voltage,three_first_peaks,three_first_peaks_index,transient_rise_gradient,P,charging_status,P_mean,P_delta,P_Plugin,P_Unplug
begin_timestamp_float,,,,,,,,,,,,,,,
1.491027e+09,701,01-04-2017_08:01:41_529732,"[7.9862e-06, 0.0259479208, -0.0389019368, 7.97...",2,"[0.6561279297, 0.7019042969, 0.7629394531, 0.7...","[289.647277832, 289.6991882324, 289.8029785156...","[26.198, 4.360, 2.076]","[754, 884, 1015]",2.458070e+05,"[105.77301788330078, 104.60313415527344, 105.3...",0,105,0,0,0
1.491027e+09,806,01-04-2017_08:14:53_521148,"[0.3013579845, 0.27552866940000004, 0.30145740...",2,"[0.732421875, 0.7171630859, 0.7476806641, 0.76...","[289.1282348633, 289.2579956055, 289.335845947...","[21.480, 3.592, 1.415]","[860, 1074, 1427]",1.951086e+05,"[105.6083755493164, 105.91771697998047, 105.88...",0,105,0,0,0
1.491073e+09,5717,01-04-2017_20:57:48_799036,"[-0.0624732189, -0.1910434365, 0.0152767645, -...",2,"[-1.2969970703, -1.2512207031, -1.2359619141, ...","[-292.2813720703, -292.3981628418, -292.618743...","[26.096, 4.280, 2.540]","[5769, 5892, 6032]",2.423178e+05,"[107.5003433227539, 107.1847915649414, 107.303...",0,106,-1,0,0
1.488462e+09,2750,02-03-2017_14:42:56_465712,"[-0.1300312473, 7.76288e-05, -0.0388483592, 0....",1,"[-2.6092529297000002, -2.6092529297000002, -2....","[0.1690305471, -0.1430258453, -0.3770681620000...","[58.447, 13.698, 33.141]","[2760, 2843, 3400]",-2.846197e+06,"[3704.182861328125, 3712.759521484375, 3715.73...",1,4850,3551,1,0
1.488462e+09,9697,02-03-2017_14:45:31_804547,"[0.142554611, 0.0336101092, 0.1024524048000000...",3,"[-31.0211181641, -31.1126708984, -31.127929687...","[-297.951385498, -297.7366638184, -297.6964111...","[40.280, 21.233, 2.845]","[9709, 9791, 9874]",-1.692315e+06,"[7219.86328125, 7217.51416015625, 7218.6166992...",1,8313,3503,1,0


## Preprossing Data

The next step is to Preprocessing the Data. In this present Case it's possible that the transient detect only on one phase, two phases or all phases. Therefore it's important to ascertain how often one of the three events is detecteted. Then we can create seperate Dataframe for this three events.

In [10]:
test = data.raw_signal_voltage.apply(lambda x: np.asarray(x))

In [11]:
test.iloc[0]

array([ 289.64727783,  289.69918823,  289.80297852, ...,  289.24502563,
        289.28396606,  289.36181641])

In [12]:
%%time
from collections import Counter
#documentation : https://docs.python.org/2/library/collections.html

# put all timestamps to a list
# and create a dict with Counter() to get a timestamp as key and a value how often we can find this timestamp
timestamps = data.index.tolist()
counter_timestamps = Counter()
for timestamp in timestamps:
    counter_timestamps[timestamp] += 1

# create a series from the counter object, that we can handle better the information
counter_timestamps_series = pd.Series(counter_timestamps)
data_point_num = len(counter_timestamps_series)
data_points_per_phase = counter_timestamps_series.value_counts()

# print all the result
print("All events: {}".format(data_point_num))
print("Events with one phase: {}".format(data_points_per_phase[1]))
print("Events with two phases: {}".format(data_points_per_phase[2]))
print("Events with three phases: {}".format(data_points_per_phase[3]))

All events: 1568
Events with one phase: 1305
Events with two phases: 212
Events with three phases: 51
CPU times: user 4.57 ms, sys: 869 µs, total: 5.44 ms
Wall time: 4.82 ms


In [13]:
# import a function that can seperate trasients by event types and extract some features
from SEPERATE_DF import *

In [14]:
%time events_from_outside, transients_1_phase, transients_2_phase, transients_3_phase = seperate_transients(data)

CPU times: user 9.41 s, sys: 42.2 ms, total: 9.45 s
Wall time: 9.46 s


In [15]:
# export df to csv files
events_from_outside.to_pickle("events_from_outside.pkl")
transients_1_phase.to_pickle("transients_1_phase.pkl")
transients_2_phase.to_pickle("transients_2_phase.pkl")
transients_3_phase.to_pickle("transients_3_phase.pkl")

In [18]:
df1 = pd.read_pickle('transients_3_phase.pkl')
df1.head()

,begin_index,begin_timestamp_string,filtered_signal,phase_num,raw_signal_current,raw_signal_voltage,three_first_peaks,three_first_peaks_index,transient_rise_gradient,P,charging_status,P_mean,P_delta,P_Plugin,P_Unplug
1.491395e+09,4016,05-04-2017_14:23:22_406877,"[0.18232499060000001, 0.1427274048, 0.18223194...",1,"[-1.9378662109, -2.0141601562, -2.0141601562, ...","[0.0390070491, -0.1430258453, -0.4550822377, -...","[22.142, 7.970, 18.781]","[4023, 4042, 4779]",1.535173e+06,"[193.2838592529297, 191.64263916015625, 194.01...",1,4914,3543,1,0
1.491395e+09,4016,05-04-2017_14:23:22_406877,"[0.3145695329, 0.3665215075, 0.3665425181, 0.3...",2,"[20.5841064453, 20.6451416016, 20.6909179688, ...","[290.9189147949, 290.9967651367, 291.061645507...","[24.063, 7.688, 16.598]","[4023, 4107, 4781]",1.677861e+06,"[3659.302490234375, 3659.780029296875, 3660.46...",1,4914,3543,1,0
1.491395e+09,4016,05-04-2017_14:23:22_406877,"[-0.016060669, 0.06456168, 0.1584541202, 0.158...",3,"[-15.4571533203, -15.4113769531, -15.380859375...","[-294.582824707, -294.4351806641, -294.3949279...","[69.691, 21.316, 45.636]","[4024, 4052, 4782]",4.173877e+06,"[3698.833740234375, 3698.317138671875, 3701.89...",1,4914,3543,1,0
1.488965e+09,3508,08-03-2017_10:27:50_128345,"[0.15550541880000002, 0.15653982760000001, 0.2...",1,"[-1.953125, -1.953125, -1.9226074219, -1.89208...","[-0.013002350000000001, -0.2080375999, -0.5200...","[23.521, 9.473, 20.440]","[3516, 3535, 4332]",1.466010e+06,"[199.44705200195312, 189.89479064941406, 200.9...",1,4866,3477,1,0
1.488965e+09,3508,08-03-2017_10:27:50_128345,"[0.2236295193, 0.28847584130000004, 0.36641946...",2,"[20.2789306641, 20.3094482422, 20.3399658203, ...","[289.1542053223, 289.2450256348, 289.309906005...","[25.649, 9.155, 21.862]","[3516, 3561, 4331]",1.593177e+06,"[3603.539794921875, 3602.707275390625, 3602.30...",1,4866,3477,1,0


In [ ]:
transients_1_phase.head()

In [ ]:
len(events_from_outside)

In [ ]:
len(transients_1_phase)

In [ ]:
len(transients_2_phase)

In [ ]:
len(transients_3_phase)

### Create Feature Dataframe from exist features

The Problem with time series is, that they are high dimensional. Therefore it's important to extract features from this time series to reduce the dimensions of the data. So the next step is to preprocessing our data, that we can extract features from this Dataframe. We will create a Dataframe for the current/voltage signals.

## Feature for one phase events

In [ ]:
from CREATE_DF import *

In [ ]:
%time feature_df_with_nan_1 = create_feature_df(transients_1_phase)

In [ ]:
# sort df: The code put all NaN values in the last columns of the df
feature_df_with_nan_sort_1 = feature_df_with_nan_1.apply(squeeze_nan, axis=1)
feature_df_with_nan_sort_1.reset_index(level=0, inplace=True)

# get columns name an put them into a list
original_columns = feature_df_with_nan_sort_1.columns.tolist()
# Create new column names
new_columns = original_columns[:4] + original_columns[10:13] + original_columns[19:20] + original_columns[22:23]
# drop all columns with NaN
feature_df_1_phase = feature_df_with_nan_sort_1.dropna(axis=1)
feature_df_1_phase = feature_df_1_phase.apply(pd.to_numeric)
feature_df_1_phase.columns = new_columns

In [ ]:
feature_df_1_phase.to_csv("feature_df_1_phase.csv", columns = new_columns, index=None)

## Feature for two phase events

In [ ]:
%time feature_df_with_nan_2 = create_feature_df(transients_2_phase)

# sort df: The code put all NaN values in the last columns of the df
feature_df_with_nan_sort_2 = feature_df_with_nan_2.apply(squeeze_nan, axis=1)
feature_df_with_nan_sort_2.reset_index(level=0, inplace=True)
# get columns name an put them into a list
original_columns = feature_df_with_nan_sort_2.columns.tolist()
# Create new column names
new_columns = original_columns[:7] + original_columns[10:16] + original_columns[19:21] + original_columns[22:24]
# drop all columns with NaN
feature_df_2_phase = feature_df_with_nan_sort_2.dropna(axis=1)
feature_df_2_phase = feature_df_2_phase.apply(pd.to_numeric)
feature_df_2_phase.columns = new_columns

In [ ]:
feature_df_2_phase.to_csv("feature_df_2_phase.csv", columns = new_columns, index=None)

## Feature for three phase events

In [ ]:
%time feature_df_3_phase = create_feature_df(transients_3_phase)

In [ ]:
feature_df_3_phase.reset_index(level=0, inplace=True)

In [ ]:
feature_df_3_phase.head()

In [ ]:
feature_df_3_phase.to_csv("feature_df_3_phase.csv", index=None)

In [ ]:
test = pd.read_csv("feature_df_3_phase.csv")

In [ ]:
test.head()

# Spielen

In [ ]:
Z = linkage(feature_df, 'ward')

In [ ]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(Z, pdist(feature_df))
c

In [ ]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
fancy_dendrogram(
    Z,
    truncate_mode='lastp',
    p=7,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=10,
    max_d=16,
)
plt.show()

In [ ]:
# create trasient data frame
raw_signal_current = data["raw_signal_current"]
raw_signal_voltage = data["raw_signal_voltage"]

# create dataframes
# on row contains on time series
# column is time or dimension
df_raw_signal_current = pd.DataFrame([x for x in raw_signal_current])
df_raw_signal_voltage = pd.DataFrame([x for x in raw_signal_voltage])

In [ ]:
df_raw_signal_current.head(1)

In [ ]:
df_raw_signal_voltage.head(1)

### Clustering

### Extract Features Voltage (mit tsfresh)

Quelle Bild:
https://tsfresh.readthedocs.io/en/latest/_images/feature_extraction_process_20160815_mc_1.png

In [ ]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import FeatureExtractionSettings

extraction_settings = FeatureExtractionSettings()
extraction_settings.IMPUTE = impute    # Fill in Infs and NaNs

# transpose since tsfresh reads times series data column-wise, not row-wise
df_raw_signal_voltage_t = df_raw_signal_voltage.copy().transpose()
df_raw_signal_voltage_t_cut = df_raw_signal_voltage_t[:100]

print(df_raw_signal_voltage_t_cut.shape)
len(df_raw_signal_voltage_t_cut.columns)

In [ ]:
# rearrange voltage column-wise, not row-wise

master_df = pd.DataFrame(df_raw_signal_voltage_t_cut[0])
master_df['id'] = 0

bar = progressbar.ProgressBar()

with progressbar.ProgressBar(max_value=len(df_raw_signal_voltage_t_cut.columns)) as bar:
    for i in range(1,500): #len(df_raw_signal_voltage_t.columns)
        signal_df = pd.DataFrame(df_raw_signal_voltage_t_cut[i])
        signal_df['id'] = i
        master_df = pd.DataFrame(np.vstack([master_df, signal_df]))
        bar.update(i)

In [ ]:
master_df.shape

In [ ]:
new_master_df = master_df.dropna(how='any')
new_master_df.shape

In [ ]:
%time X = extract_features(new_master_df, column_id=1, feature_extraction_settings=extraction_settings)